In [ ]:
from PIL import Image
import numpy as np
import socket
import time
import matplotlib.pyplot as plt

In [ ]:
def read_and_pad_image(image_path, target_size=(128, 64)):
    """
    Read a PNG image and pad it to the target size.
    
    :param image_path: string, path to the image file
    :param target_size: tuple, target size for padding (width, height)
    :return: 2D numpy array of booleans
    """
    # Open the image
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    
    # Resize the image if it's larger than the target size
    if image.size[0] > target_size[0] or image.size[1] > target_size[1]:
        image.thumbnail(target_size, Image.ANTIALIAS)
    
    # Create a new image with the target size and paste the original image into it
    padded_image = Image.new('L', target_size, color=255)  # White background
    padded_image.paste(image, ((target_size[0] - image.size[0]) // 2, 
                               (target_size[1] - image.size[1]) // 2))
    
    # Convert the image to a 2D boolean array (True for black, False for white)
    bool_array = np.array(padded_image) < 128  # Assuming a threshold of 128 for binarization
    
    return bool_array

def plot_bool_array(bool_array):
    """
    Plot a 2D boolean array using matplotlib.
    
    :param bool_array: 2D numpy array of booleans
    """
    plt.imshow(bool_array, cmap='gray', interpolation='nearest')
    plt.title('2D Boolean Array')
    plt.show()

image_path = 'frames/frame_0.png'  # Replace with your image path
bool_array = read_and_pad_image(image_path)
plot_bool_array(bool_array)

In [ ]:
def bool_array_to_byte_array(bool_array):
    """
    Convert a 2D boolean array to a byte array.
    
    :param bool_array: 2D numpy array of booleans
    :return: bytearray
    """
    flat_bool_array = bool_array.flatten()
    byte_array = bytearray()
    
    for i in range(0, len(flat_bool_array), 8):
        byte = 0
        for bit in range(8):
            if i + bit < len(flat_bool_array) and flat_bool_array[i + bit]:
                byte |= (1 << bit)
        byte_array.append(byte)
    
    return byte_array


byte_array = bool_array_to_byte_array(bool_array)

print(byte_array)
print(len(byte_array))

In [ ]:
def send_bool_array(bool_array, ip_address, port, packet_size=512, array_id=0):
    """
    Send a 2D boolean array to a specified IP address and port using UDP.
    
    :param bool_array: 2D numpy array of booleans
    :param ip_address: string, target IP address
    :param port: int, target port
    :param packet_size: int, size of each packet
    :param array_id: int, id of the array
    """
    byte_array = bool_array_to_byte_array(bool_array)
    height, width = bool_array.shape
    total_size = len(byte_array)
    header_size = 5
    
    # Calculate the number of packets accounting for the header size
    num_packets = (total_size + packet_size - header_size - 1) // (packet_size - header_size)

    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    try:
        for seq_num in range(num_packets):
            start = seq_num * (packet_size - header_size)
            end = min(start + (packet_size - header_size), total_size)  # Ensure not to exceed the total size
            packet_data = byte_array[start:end]
            
            # Packet structure: array_id (1 byte), seq_num (1 byte), num_packets (1 byte), width (1 byte), height (1 byte), packet_data
            packet = bytearray([array_id, seq_num, num_packets, width, height]) + packet_data
            sock.sendto(packet, (ip_address, port))
            print(f"Sent packet {seq_num + 1}/{num_packets} of array {array_id} to {ip_address}:{port}")
    finally:
        sock.close()


ip = '192.168.1.8'
port = 5005  
packet_size = 512

for array_id in range(100):
    send_bool_array(bool_array, ip, port, packet_size=packet_size, array_id=array_id)
    time.sleep(0.05)